# Dog Breed Identification
### <span style="color:red">USE PYTHON 3.7</span>
Jupyter notebook to document and display outputs for live code.
*Currently using test code from tensorflow's image classification* found:
[here](https://www.tensorflow.org/tutorials/images/classification)

Code written here imports required libraries for function

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

placeholder

In [2]:
print("Hello, World")

Hello, World
